In [2]:
# Import necessary module
import pandas as pd
import pymysql

In [3]:
# connect with MySQL database
myconnection = pymysql.connect(host='127.0.0.1',user='root',passwd='Krisrak@123', database='Redbus_webscraped_data')

In [10]:
#Read table from database and store it in df using MySQL query
query = """select * from Redbus_webscraped_data.bus_routes;"""
df = pd.read_sql_query(query,myconnection)

C:\Users\rakes\AppData\Local\Temp\ipykernel_25936\530438284.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query,myconnection)


In [11]:
#get top 5 content/data of table
df.head()

,id,route_Departure_name,route_Arrival_name,route_link,busname,bustype,departing_time,duration,reaching_time,star_rating,price,seats_available,route_name
0,1,BAREILLY SATELITE,Anand Vihar,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - PLB0055,Rajdhani Non AC Seater 2+3,2024-07-17 08:30:00,05h 59m,2024-07-17 14:29:00,3.3,454.0,51.0,BAREILLY SATELITE - Anand Vihar
1,2,BAREILLY SATELITE,Kaushambi Bus Stand(Delhi),https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - BLY0108,Janrath AC Seater 2+2,2024-07-17 09:02:00,04h 43m,2024-07-17 13:45:00,3.3,598.0,36.0,BAREILLY SATELITE - Kaushambi Bus Stand(Delhi)
2,3,BAREILLY SATELITE,Kaushambi Bus Stand(Delhi),https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - KSB0071,Shatabdi AC Seater 2+2,2024-07-17 09:32:00,05h 28m,2024-07-17 15:00:00,3.3,598.0,36.0,BAREILLY SATELITE - Kaushambi Bus Stand(Delhi)
3,4,BAREILLY OLD BUS STATION,Anand Vihar,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - RKD0053,Ordinary Non AC Seater 2+3,2024-07-17 10:00:00,05h 30m,2024-07-17 15:30:00,3.3,448.0,52.0,BAREILLY OLD BUS STATION - Anand Vihar
4,5,BAREILLY SATELITE,Kaushambi Bus Stand(Delhi),https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - RKD0110,Shatabdi AC Seater 2+2,2024-07-17 10:01:00,05h 00m,2024-07-17 15:01:00,4.3,598.0,33.0,BAREILLY SATELITE - Kaushambi Bus Stand(Delhi)


In [8]:
#get COLUMN_NAME and DATA_TYPE of table using MySQL query
query = """SELECT COLUMN_NAME, DATA_TYPE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'bus_routes' AND TABLE_SCHEMA = 'redbus_webscraped_data';"""
pd.read_sql_query(query,myconnection)

C:\Users\rakes\AppData\Local\Temp\ipykernel_25936\820754652.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query,myconnection)


,COLUMN_NAME,DATA_TYPE
0,id,int
1,route_Departure_name,text
2,route_Arrival_name,text
3,route_link,text
4,busname,text
5,bustype,text
6,departing_time,datetime
7,duration,text
8,reaching_time,datetime
9,star_rating,float


In [4]:
#ALTER, DELETE, REPLACE and UPDATE table
""" 
-> if there null is in route_Departure_name, then query will delete those particular row only
-> if there null is in star_rating, then query will set to Null data for purpose of changing the datatype of column
-> if there Seats available,Seat available or null words is in seats_available, then query will replace those with empty string
-> after these step, ALTER used for update data type of table
"""
queries = [
    "SET sql_safe_updates = 0;",
    "DELETE FROM Redbus_webscraped_data.bus_routes WHERE route_Departure_name = 'null';",
    "UPDATE Redbus_webscraped_data.bus_routes SET star_rating = NULL WHERE star_rating = 'null';",
    "UPDATE Redbus_webscraped_data.bus_routes SET seats_available = REPLACE(seats_available, 'Seats available', '');",
    "UPDATE Redbus_webscraped_data.bus_routes SET seats_available = REPLACE(seats_available, 'Seat available', '');",
    "UPDATE Redbus_webscraped_data.bus_routes SET seats_available = NULL WHERE seats_available = 'null';",
    "UPDATE Redbus_webscraped_data.bus_routes SET departing_time = STR_TO_DATE(departing_time, '%H:%i');",
    "ALTER TABLE Redbus_webscraped_data.bus_routes MODIFY departing_time TIME;",
    "ALTER TABLE Redbus_webscraped_data.bus_routes MODIFY departing_time DATETIME;",
    "UPDATE Redbus_webscraped_data.bus_routes SET reaching_time = STR_TO_DATE(reaching_time, '%H:%i');",
    "ALTER TABLE Redbus_webscraped_data.bus_routes MODIFY reaching_time TIME;",
    "ALTER TABLE Redbus_webscraped_data.bus_routes MODIFY reaching_time DATETIME;",
    "ALTER TABLE Redbus_webscraped_data.bus_routes MODIFY star_rating FLOAT;",
    "ALTER TABLE Redbus_webscraped_data.bus_routes MODIFY price DECIMAL(10, 2);",
    "ALTER TABLE Redbus_webscraped_data.bus_routes MODIFY seats_available INT;"
]

# Execute each above query individually
with myconnection.cursor() as cursor:
    for query in queries:
        cursor.execute(query)
        myconnection.commit()

In [9]:
# add route_name column in table using ALTER and UPDATE
# sql_safe_updates using for update the table
queries_to_add = [
    'SET sql_safe_updates = 0;',
    'ALTER table Redbus_webscraped_data.bus_routes add column route_name text;',
    'UPDATE Redbus_webscraped_data.bus_routes set route_name = concat(route_Departure_name, " - ", route_Arrival_name);'
]

# Execute each above query individually
with myconnection.cursor() as cursor:
    try:
        for query in queries_to_add:
            cursor.execute(query)
            myconnection.commit()
    except Exception as e:
        print(e)
        myconnection.rollback()
